In [ ]:
#FCN feature

import numpy as np
import torch
import cv2
from FCN import *

def fcn(img_folder):
    img_path = img_folder
    image = cv2.imread(img_path)
    h, w, ch = image.shape
    n_class = 20
    batch_size = 10

    # 对图像内容计算
    input = image.transpose((2, 0, 1))
    input = np.float32(input) / 255.0
    input = np.reshape(input, [1, ch, h, w])
    input = torch.from_numpy(input)
    # input = input.cuda(gpu_id)

    print(type(input))
    vgg_model = VGGNet(requires_grad=True)
    fcn_model = FCNs(pretrained_net=vgg_model, n_class=n_class)
    output = fcn_model(input)
    out = output.data.numpy()
    
    h = h - 1
    w = w - 1
    feat = np.zeros((h, w))
    for i in range(h):
        for j in range(w):
            feat[i, j] = np.mean(out[0, :, i, j])

    return feat

In [ ]:
import skimage
from skimage.segmentation import felzenszwalb, mark_boundaries, slic
from os.path import basename, join, isfile
import os
import matplotlib.pyplot as plt    

import torchvision.models as models

%matplotlib inline

basename = os.getcwd()
img_dir = join(basename, 'input')
img_list = [join(img_dir, f) for f in os.listdir(img_dir) if isfile(join(img_dir, f))]

# vis part
for img_path in img_list:
    
    if img_path[-3:] == 'jpg':
        
        #  get conv feature
        img = cv2.imread(img_path)
#         conv = fcn(img_path).astype(np.double) 
        conv = fcn(img_path)
        print(conv.shape)
        
        plt.figure()
        plt.imshow(conv, cmap='gray')
        plt.show()
        
        gray_img = skimage.io.imread(img_path, as_gray=True)
        gpu_id = 0
#         conv.cuda(gpu_id)
        comb = (gray_img[1:,1:] + conv) / 2
        

           # sp method: felzenszwalb
        # 1. use rgb
        segments_rgb = felzenszwalb(img, scale=100, sigma=0.5, min_size=50)
        # 2. use vgg16 conv_1, conv_2, conv_1_2
        segments_conv = felzenszwalb(conv, scale=100, sigma=0.5, min_size=50)
        # 3. combined?
        segments_comb = felzenszwalb(comb, scale=100, sigma=0.5, min_size=50)
        
                
#         # sp method: SLIC
#         # 1. use img
#         segments_rgb = slic(img, n_segments=250, compactness=10, sigma=1)
#         # 2. use feat_1, feat_2, feat_1_2
#         segments_conv = slic(conv, n_segments=50, compactness=10, sigma=1)
#         # 3. combined?
#         segments_comb = slic(comb, n_segments=250, compactness=10, sigma=1)

        
        # show superpixels results
        fig, ax = plt.subplots(3, 2, figsize=(10, 10), sharex=True, sharey=True)
        


        # superpixel vis
        ax[0, 0].imshow(img)
        ax[0, 0].set_title('ori-img')
   
        ax[0, 1].imshow(mark_boundaries(img[1:, 1:],segments_conv))
        ax[0, 1].set_title('conv_1_feat')
        
        ax[1, 0].imshow(mark_boundaries(img, segments_rgb))
        ax[1, 0].set_title('rgb_feat')
        
        ax[1, 1].imshow(mark_boundaries(img[1:, 1:], segments_comb))
        ax[1, 1].set_title('comb_feat')
        plt.show()        
